In [4]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 4.7 MB/s eta 0:00:00


In [11]:
# Faltantes por OPS — inclui também OPS sem ausências (perc_* = 0)
# Requisitos: pandas, openpyxl
import pandas as pd, numpy as np, os, re
from typing import Optional, List

# ---------------- PARÂMETROS ----------------
ARQUIVO_DADOS = "dados_2001_2008.xlsx"
ANOS = list(range(2001, 2009))
PASTA_SAIDA = "./saidas_faltantes_2001_2008"
os.makedirs(PASTA_SAIDA, exist_ok=True)

# Faixas por posição no Excel
FAIXA_CONTABEIS = ("B", "AZ")
FAIXA_INDICADORES_INICIO = "BA"   # BA até o fim
FAIXA_INDICADORES_MODELO = ("BB", "CR")

# Listar só quem tem faltante?  -> False para incluir todos
filtrar_somente_faltantes = False

# --------------- HELPERS --------------------
def excel_col_to_idx(col_str: str) -> int:
    col_str = col_str.strip().upper()
    idx = 0
    for ch in col_str:
        idx = idx * 26 + (ord(ch) - ord('A') + 1)
    return idx - 1

def normalize_cols(df: pd.DataFrame) -> pd.DataFrame:
    mapping = {c: re.sub(r"\s+", " ", str(c)).strip().lower() for c in df.columns}
    return df.rename(columns=mapping)

def guess_ops_col(df_norm: pd.DataFrame) -> str:
    candidatos = ["ops","id_ops","codigo_ops","cod_ops","código ops","codigo da ops"]
    for c in candidatos:
        if c in df_norm.columns:
            return c
    return df_norm.columns[0]

def cols_slice_by_letters(df: pd.DataFrame, ini: str, fim: Optional[str]=None) -> List[str]:
    n = df.shape[1]
    i0 = excel_col_to_idx(ini)
    i1 = excel_col_to_idx(fim) if fim is not None else n - 1
    i0 = max(0, i0); i1 = min(n - 1, i1)
    if i0 > i1 or i0 >= n:
        return []
    return list(df.columns[i0:i1+1])

def contar_faltantes(df_use: pd.DataFrame, col_ops: str) -> pd.DataFrame:
    """
    Calcula contagens e percentuais de faltantes por linha (OPS) para:
      - total (tudo menos OPS)
      - contábeis (B:AZ)
      - indicadores (BA:última)
      - indicadores do modelo (BB:CR)
    SEM FILTRAR linhas com zero faltantes (ou seja, inclui todas as OPS).
    """
    if col_ops not in df_use.columns:
        raise KeyError(f"Coluna OPS '{col_ops}' não está no DataFrame recebido.")

    # Listas de colunas por faixa (posição Excel; df_use está NORMALIZADO e mantém a ordem)
    all_cols      = [c for c in df_use.columns if c != col_ops]
    cont_cols     = [c for c in cols_slice_by_letters(df_use, *FAIXA_CONTABEIS) if c != col_ops]
    ind_cols      = [c for c in cols_slice_by_letters(df_use, FAIXA_INDICADORES_INICIO, None) if c != col_ops]
    ind_mod_cols  = [c for c in cols_slice_by_letters(df_use, *FAIXA_INDICADORES_MODELO) if c != col_ops]

    # Contagens de faltantes por linha (preenchem 0 quando lista vazia)
    falt_total = df_use[all_cols].isna().sum(axis=1) if all_cols else pd.Series(0, index=df_use.index)
    falt_cont  = df_use[cont_cols].isna().sum(axis=1) if cont_cols else pd.Series(0, index=df_use.index)
    falt_ind   = df_use[ind_cols].isna().sum(axis=1) if ind_cols else pd.Series(0, index=df_use.index)
    falt_ind_m = df_use[ind_mod_cols].isna().sum(axis=1) if ind_mod_cols else pd.Series(0, index=df_use.index)

    # Quantidade de colunas por grupo (para %)
    n_total, n_cont, n_ind, n_ind_m = len(all_cols), len(cont_cols), len(ind_cols), len(ind_mod_cols)

    out = pd.DataFrame({
        "ops": df_use[col_ops].astype(str).values,
        "falt_total": falt_total.values,
        "falt_cont":  falt_cont.values,
        "falt_ind":   falt_ind.values,
        "falt_ind_modelo": falt_ind_m.values,
        "cols_total": n_total,
        "cols_cont":  n_cont,
        "cols_ind":   n_ind,
        "cols_ind_modelo": n_ind_m,
    })

    # Percentuais (0 quando não há colunas na faixa)
    out["perc_total"]        = out["falt_total"]        / n_total if n_total  > 0 else 0.0
    out["perc_cont"]         = out["falt_cont"]         / n_cont  if n_cont   > 0 else 0.0
    out["perc_ind"]          = out["falt_ind"]          / n_ind   if n_ind    > 0 else 0.0
    out["perc_ind_modelo"]   = out["falt_ind_modelo"]   / n_ind_m if n_ind_m  > 0 else 0.0

    # NÃO filtra zero-faltantes; apenas ordena para diagnóstico
    out = out.sort_values(["perc_total","perc_cont","perc_ind"], ascending=[False, False, False]).reset_index(drop=True)
    return out

# --------------- EXECUÇÃO --------------------
xl = pd.ExcelFile(ARQUIVO_DADOS)
registros = []

for ano in ANOS:
    aba = str(ano)
    if aba not in xl.sheet_names:
        print(f"[AVISO] Aba {aba} ausente."); continue

    df_raw = xl.parse(aba, header=0)
    if df_raw.empty:
        print(f"[AVISO] Aba {aba} vazia."); continue

    # Usar df normalizado (nomes em minúsculas e estáveis)
    df_norm = normalize_cols(df_raw.copy())
    col_ops = guess_ops_col(df_norm)

    out = contar_faltantes(df_norm, col_ops=col_ops)
    out.insert(1, "ano", ano)

    # Se quiser somente quem tem algum faltante, ligue o filtro abaixo:
    if filtrar_somente_faltantes:
        mask = (out[["falt_total","falt_cont","falt_ind","falt_ind_modelo"]].sum(axis=1) > 0)
        out = out.loc[mask].reset_index(drop=True)

    registros.append(out)
    out.to_csv(os.path.join(PASTA_SAIDA, f"faltantes_por_ops_{ano}.csv"), index=False, encoding="utf-8")

# Consolidação
if registros:
    base = pd.concat(registros, ignore_index=True)
    base.to_csv(os.path.join(PASTA_SAIDA, "faltantes_por_ops_2001_2008.csv"), index=False, encoding="utf-8")

    # Agregado por OPS (médias dos percentuais ao longo dos anos)
    agg = (base.groupby("ops", as_index=False)
                .agg(anos_observados=("ano","nunique"),
                     falt_total_medio=("perc_total","mean"),
                     falt_cont_medio=("perc_cont","mean"),
                     falt_ind_medio=("perc_ind","mean"),
                     falt_ind_modelo_medio=("perc_ind_modelo","mean"))
                .sort_values("falt_total_medio", ascending=False))
    agg.to_csv(os.path.join(PASTA_SAIDA, "faltantes_agregado_por_ops.csv"), index=False, encoding="utf-8")

print("OK: faltantes gerados em", PASTA_SAIDA)


OK: faltantes gerados em ./saidas_faltantes_2001_2008
